In [1]:
import numpy as np
import cv2
import time
import glob

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import dv

from joblib import dump, load

In [2]:
def loadLabel(path):
    labels = np.genfromtxt(path, delimiter=',', skip_header=1, dtype=int)

    start_time = labels[0][1]
    for i in labels:
        i[1] = i[1]-start_time
        i[2] = i[2]-start_time
        
    return labels

In [3]:
# Return the start time of the given class index.
def start(index, labels):
    return labels[index][1]
 
# Return the end time of the given class index.
def end(index, labels):
    return labels[index][2]

def classID(index, labels):
    return labels[index][0]-1

# Return the class of the current frame. If no gesture then return -1.
def setGesture(currentTime, labels):
    for i in range(len(labels)-1):
        if (currentTime >= start(i, labels) and currentTime <= end(i, labels)):
            return classID(i, labels)
    return -1

In [4]:
def process(X, Y, path, labels):
    offset = 0
    
    with dv.AedatFile(path) as f: 
        for frame in f['frames']:    
            # Set the offset on the first frame.
            if (offset == 0):
                offset = frame.timestamp

            gesture = setGesture((frame.timestamp - offset), labels)
            image = frame.image

            # font
            font = cv2.FONT_HERSHEY_SIMPLEX

            # org
            org = (5, 30)

            # fontScale
            fontScale = 1

            # Blue color in BGR
            color = (255, 0, 0)

            # Line thickness of 2 px
            thickness = 2
            image = cv2.putText(image, str(gesture), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
            
            # cv2.imshow('out',image)
            # cv2.waitKey(1)

            if (gesture == -1):
                continue
            else:
                Y.append(gesture)

                # TODO: Scale the data.
                matrix = frame.image.flatten()
                X.append(matrix)

In [5]:
def verify(csv, file):
    
    csv = csv.split("_labels")
    file = file.split(".")
    
    if (csv[0] != file[0]):
        print("mismatch: (%s, %s)" % (csv[0], file[0]))

In [6]:
X_test = []
Y_test = []

split = 98 

csv = sorted(glob.glob("data/*.csv"))
aedat4 = sorted(glob.glob("data/*.aedat4"))

for i in range(split, len(csv)):
    verify(csv[i], aedat4[i])
    print(csv[i], aedat4[i])
    
    labels = loadLabel(csv[i])
    process(X_test, Y_test, aedat4[i], labels)

data\user24_fluorescent_labels.csv data\user24_fluorescent.aedat4
data\user24_fluorescent_led_labels.csv data\user24_fluorescent_led.aedat4
data\user24_led_labels.csv data\user24_led.aedat4
data\user25_fluorescent_labels.csv data\user25_fluorescent.aedat4
data\user25_led_labels.csv data\user25_led.aedat4
data\user26_fluorescent_labels.csv data\user26_fluorescent.aedat4
data\user26_fluorescent_led_labels.csv data\user26_fluorescent_led.aedat4
data\user26_lab_labels.csv data\user26_lab.aedat4
data\user26_led_labels.csv data\user26_led.aedat4
data\user26_natural_labels.csv data\user26_natural.aedat4
data\user27_fluorescent_labels.csv data\user27_fluorescent.aedat4
data\user27_fluorescent_led_labels.csv data\user27_fluorescent_led.aedat4
data\user27_led_labels.csv data\user27_led.aedat4
data\user27_natural_labels.csv data\user27_natural.aedat4
data\user28_fluorescent_labels.csv data\user28_fluorescent.aedat4
data\user28_fluorescent_led_labels.csv data\user28_fluorescent_led.aedat4
data\use

In [7]:
pipeline = load('pipeline.joblib')

In [8]:
pipeline.score(X_test, Y_test)

0.8062078712763644